In [1]:
import os
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose

In [14]:
path = '../Data/Preprocessed/탈모톡톡_탈모수다.xlsx'

df_origin = pd.read_excel(path)
df_origin['date'] = pd.to_datetime(df_origin['date'])
df_origin.loc[df_origin['rank'].isnull(), 'rank'] = 'any'
df_origin.head(2)

column_list = ['Fuck탈모', '고민상담', '나의치료법', '일반잡담', '주제토론', '탈모제품']

In [7]:
df_origin['category'].unique()

array([nan, '헤어라인', 'Fuck탈모', '고민상담', '주제토론', '일반잡담', '나의치료법', 'DUPA토론',
       '해외뉴스', '처방정보', '탈모진단', '탈모x두피고민', '넓은이마', '남자증모술', '탈모제품', 'Q&A',
       '탈모인증', '공지'], dtype=object)

In [8]:
df_origin['rank'].unique()

array(['any', 6, 4, 16, 13, 2, 19, 14, 12, 15, 5, 23, 10, 3, 21, 9, 25,
       24, 28, 11, 17, 18, 30, 20, 29, 8, 22, 1, 26, 31, 50, 33, 32, 37,
       46, 'admin', 39, 27], dtype=object)

In [16]:
# 탈모수다 게시글: 년도별 게시글 건수
df = df_origin.copy()
df.set_index('date', inplace = True)
df = df[df['category'].apply(lambda x : x in column_list)]
result = df.groupby(['category']).resample('1Y')['href'].count().reset_index().pivot_table(index = ['date'], columns = ['category'], values = ['href']).fillna(0)
result

href                                     
category    Fuck탈모    고민상담  나의치료법    일반잡담   주제토론   탈모제품
date                                                   
2019-12-31  2313.0  1687.0   97.0   277.0  229.0    0.0
2020-12-31  9577.0  6744.0  493.0  1190.0  817.0    0.0
2021-12-31  7326.0  5220.0  412.0   813.0  674.0   88.0
2022-12-31  5166.0  3542.0    0.0     0.0  598.0  597.0
2023-12-31  2505.0  2055.0    0.0     0.0  360.0  299.0

In [13]:
# 탈모수다 게시글: category별 총 게시글 수
df = df_origin.copy()
result = df.groupby(['category'])['href'].count().reset_index()
result['href'] = result['href'].apply(lambda x : round(x / result['href'].sum() * 100, 2))
result.sort_values(by = ['href'], ascending = False)

,category,href
1,Fuck탈모,49.62
3,고민상담,35.52
9,주제토론,4.94
8,일반잡담,4.21
5,나의치료법,1.85
13,탈모제품,1.82
14,탈모진단,1.33
12,탈모인증,0.28
0,DUPA토론,0.21
2,Q&A,0.11


In [24]:
# 탈모수다 게시글: 주요 category 및 연도별 평균 조회수
df = df_origin.copy()
df = df[df['category'].isin(column_list)]
df.set_index('date', inplace = True)
result = df.groupby(['category']).resample('Q')['view'].mean().reset_index()
result['view'] = result['view'].apply(lambda x : round(x))
result.pivot_table(columns = ['category'], index = ['date'], values = ['view']).fillna(0)

view                                        
category    Fuck탈모    고민상담   나의치료법    일반잡담    주제토론    탈모제품
date                                                      
2019-12-31  1769.0  1664.0  2878.0  1442.0  1721.0     0.0
2020-03-31  1509.0  1451.0  2913.0  1121.0  1564.0     0.0
2020-06-30  1561.0  1220.0  3816.0  1347.0  1766.0     0.0
2020-09-30  1508.0  1228.0  3448.0  1238.0  1798.0     0.0
2020-12-31  1891.0  1496.0  4087.0  1075.0  1714.0     0.0
2021-03-31  1983.0  1988.0  3332.0  1122.0  2339.0     0.0
2021-06-30  2346.0  1711.0  2847.0  1864.0  1896.0     0.0
2021-09-30  1808.0  1237.0  3253.0  1270.0  2183.0     0.0
2021-12-31  1891.0  1261.0  2007.0  1535.0  2361.0  1545.0
2022-03-31  1488.0  1281.0     0.0     0.0  1210.0   988.0
2022-06-30   910.0   771.0     0.0     0.0   729.0   863.0
2022-09-30  1282.0  1038.0     0.0     0.0  1011.0  1085.0
2022-12-31   847.0   714.0     0.0     0.0   555.0   611.0
2023-03-31  1621.0  1246.0     0.0     0.0   923.0   939.0
2023-06-30   882.0   687.0     0.0     0.0   729.0   782.0
2023-09-30   634.0   570.0     0.0     0.0   564.0   706.0

In [29]:
# 탈모수다 게시글: 주요 category 및 rank별 게시글 수
df = df_origin.copy()
df = df[df['category'].isin(column_list)]
result = df.groupby(['category', 'rank'])['href'].count().reset_index()
result = result.pivot_table(index = ['rank'], columns = ['category'], values = ['href']).fillna(0)

for i in range(len(result.columns)):
    result.iloc[:, i] = round(result.iloc[:, i] / result.iloc[:, i].sum() * 100)

result

href                              
category Fuck탈모  고민상담 나의치료법  일반잡담  주제토론  탈모제품
rank                                         
1           0.0   0.0   0.0   0.0   0.0   1.0
2           4.0   7.0   7.0   6.0   2.0   3.0
3           5.0   8.0  13.0   8.0   4.0   6.0
4          16.0  22.0  25.0  23.0  11.0  22.0
5          13.0  14.0  12.0  11.0  12.0  20.0
6           7.0   7.0   7.0   6.0   7.0  10.0
8           1.0   1.0   1.0   1.0   1.0   0.0
9           7.0   5.0   5.0   4.0   6.0   7.0
10          4.0   3.0   4.0   3.0   4.0   4.0
11          3.0   2.0   3.0   2.0   3.0   2.0
12          2.0   1.0   2.0   2.0   4.0   2.0
13          2.0   2.0   2.0   3.0   2.0   1.0
14          2.0   1.0   1.0   2.0   2.0   1.0
15          2.0   1.0   1.0   1.0   2.0   0.0
16          1.0   1.0   0.0   2.0   1.0   0.0
17          1.0   0.0   1.0   3.0   1.0   0.0
18          1.0   1.0   2.0   3.0   1.0   0.0
19          3.0   1.0   1.0   2.0   4.0   1.0
20          1.0   1.0   1.0   1.0   3.0   0.0
21          1.0   0.0   2.0   1.0   2.0   1.0
22          1.0   0.0   0.0   1.0   3.0   1.0
23          1.0   0.0   0.0   0.0   1.0   0.0
24          0.0   0.0   0.0   1.0   1.0   1.0
25          0.0   1.0   0.0   1.0   1.0   0.0
26          0.0   0.0   0.0   1.0   0.0   0.0
27          0.0   0.0   0.0   0.0   0.0   0.0
28          1.0   0.0   0.0   0.0   0.0   0.0
29          0.0   0.0   0.0   0.0   0.0   0.0
30          0.0   0.0   0.0   0.0   0.0   0.0
31          0.0   0.0   1.0   0.0   0.0   0.0
32          0.0   0.0   0.0   0.0   0.0   0.0
33          0.0   0.0   0.0   0.0   0.0   0.0
37          0.0   0.0   0.0   0.0   0.0   0.0
39          0.0   0.0   0.0   0.0   0.0   0.0
46          0.0   0.0   0.0   0.0   0.0   0.0
50          0.0   0.0   0.0   0.0   0.0   0.0
admin       0.0   0.0   0.0   0.0   0.0   0.0
any        19.0  21.0   9.0  13.0  24.0  15.0

In [32]:
# 탈모수다 게시글: 주요 category 및 rank별 평균 조회수
df = df_origin.copy()
df = df[df['category'].isin(column_list)]
result = df.groupby(['category', 'rank'])['view'].mean().reset_index()
result = result.pivot_table(index = ['rank'], columns = ['category'], values = ['view']).fillna(0)
result = result.apply(lambda x : round(x))
result

result

view                                          
category  Fuck탈모     고민상담    나의치료법    일반잡담    주제토론    탈모제품
rank                                                      
1         1399.0   1108.0   1776.0   833.0   672.0   568.0
2         1225.0   1068.0   1353.0   931.0  1220.0   883.0
3         1238.0   1220.0   1655.0  1269.0  1134.0   934.0
4         1134.0   1070.0   1633.0  1003.0   896.0   762.0
5         1369.0   1289.0   2878.0  1400.0  1351.0   872.0
6         1614.0   1298.0   3754.0  1559.0  1168.0  1043.0
8         1919.0   1414.0   5628.0  1779.0  1583.0   668.0
9         1816.0   1521.0   4896.0  2157.0  1609.0  1235.0
10        1953.0   1593.0   9418.0  1447.0  1601.0  1042.0
11        2136.0   1943.0   4799.0  1235.0  1899.0  1685.0
12        2182.0   1508.0   5303.0  1564.0  1550.0   983.0
13        2346.0   1607.0   6622.0  1071.0  1442.0   939.0
14        2087.0   1292.0   3732.0   962.0  1331.0   731.0
15        2135.0   1319.0   2735.0  1120.0  1667.0   862.0
16        1437.0   1662.0   2103.0   768.0  1417.0  2105.0
17        1503.0   1627.0   8609.0   771.0  1577.0  3840.0
18        1270.0   1614.0   4737.0   738.0  1180.0  1095.0
19        1876.0   1370.0   4467.0  1791.0  1404.0  2754.0
20        2692.0   1743.0   3799.0  2768.0  4333.0   859.0
21        2253.0   1377.0   6584.0  3972.0  2770.0   862.0
22        2444.0   1843.0   6707.0  1975.0  2934.0   941.0
23        1531.0    994.0  17680.0   824.0  1257.0   746.0
24        1387.0   1524.0      0.0  1014.0  1249.0  1520.0
25        1750.0   2033.0   2581.0  2070.0  2089.0   985.0
26        3771.0    799.0      0.0   573.0     0.0     0.0
27        1338.0  16523.0      0.0     0.0     0.0   694.0
28        4051.0   1473.0   1043.0  2898.0   829.0     0.0
29        1209.0   1088.0      0.0  1447.0     0.0     0.0
30           0.0      0.0      0.0  1474.0     0.0     0.0
31        4342.0   2058.0   6189.0  1238.0  4164.0   403.0
32        1611.0      0.0      0.0     0.0  1256.0  1304.0
33        2174.0    334.0      0.0   837.0  1268.0     0.0
37        1348.0      0.0      0.0   600.0  4919.0     0.0
39        2068.0      0.0   1551.0  5494.0   728.0  2902.0
46        1086.0   1523.0      0.0     0.0  1283.0   838.0
50        1203.0      0.0      0.0     0.0     0.0     0.0
admin      505.0      0.0      0.0     0.0     0.0  1580.0
any       1533.0   1400.0   3428.0  1442.0  1438.0   739.0